# Act102 - Mathématiques actuarielles fondamentales de l'assurance vie

## Cours Table de Mortalité - Partie 2

Guillaume Gorge 2022
Cours du CNAM

<!-- https://www.markdownguide.org/basic-syntax/ -->


# On l'on essaye de créer une table de mortalité prédictive

#### Initialisation du système python

In [1]:
#import sys
#Stat Panda
#!{sys.executable} -m pip install pandas_profiling 
#Stat Assurance vie
#!{sys.executable} -m pip install lifelines    
#!{sys.executable} -m pip install seaborn  
#!{sys.executable} -m pip install xlrd
#!{sys.executable} -m pip install plotnine
#pip install pylexis

In [12]:
import os
!pip install -q xlrd
import pandas as pd

# packages classiques pour gérer des données
import pandas as pd
import numpy as np

# graphiques
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 
#option pour que les graphes apparaissent dans Jupyter

# Stat sur Panda
import pandas_profiling 


In [3]:
#option pour que les graphes apparaissent dans Jupyter
%matplotlib inline 

#### Adresse des répertoires
Adresse à personnaliser selon environnement

In [9]:
BASE_PATH = 'C:/Users/guill/GitHub/RessourcesActuariat/ActuariatVieCNAM/DataActuariat'
BASE_PATH = 'https://github.com/RessourcesActuariat/ActuariatVieCNAM/tree/main/DataActuariat'
# image and mask directories
DataName = f'{BASE_PATH}/FD_DEC_2019.csv'
VarName = f'{BASE_PATH}/varmod_DEC_2019.csv'
DataNameExpo = f'{BASE_PATH}/pop-totale-france-metro.xls'
DataNameExpo='https://github.com/RessourcesActuariat/ActuariatVieCNAM/blob/main/DataActuariat/pop-totale-france-metro.xls?raw=true'

In [10]:
from google.colab import files
file_id = '1Sv4ib5i7CKWhAHZkKg-uitIkS3xwxtXM'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('exported.xlsx')

In [14]:
uploaded = files.upload()

Saving FD_DEC_2019.csv to FD_DEC_2019.csv
Saving pop-totale-france-metro.xls to pop-totale-france-metro.xls
Saving varmod_DEC_2019.csv to varmod_DEC_2019.csv


In [15]:
uploaded


In [13]:
EXPOData = pd.read_excel(DataNameExpo)  
# A noter, nous avons fait quelques retraitements de forme sur le fichier pour simplifier sa lecture#EXPOData.head()

ImportError: ignored

In [ ]:
# Fichier fourni par l'INSEE avec toutes les modalités
VarDataName = pd.read_csv(VarName,sep=';')  # dictionnaire des variables
# Fonction pour imprimer les modalités
def ImprimeModalite(feature,dfName = VarDataName):
    Mod = dfName.loc[dfName['COD_VAR']==feature]
    display(Mod[['COD_MOD','LIB_MOD']])


In [ ]:
DEATHData=pd.read_csv(DataName,sep=';')  


In [ ]:
DEATHSynthese = DEATHData.groupby(["SEXE","ANAIS"])['SEXE'].count().rename('NombreMorts').to_frame()

In [ ]:
EXPOMORTData = pd.merge(EXPOData[['SEXE','ANAIS','AGE','EXPOSITION','CENSURE']], DEATHSynthese, on=['SEXE','ANAIS'], how='left')
display(EXPOMORTData)

In [ ]:
def CalculPxMethode2(df=EXPOMORTData):
    df['Lx'] = df['EXPOSITION']
    df['Lx1'] = df['EXPOSITION'] - df['NombreMorts']
        # Selon la definition de Px
    df['Px'] = df['Lx1']/df['Lx'] 
    df['Qx'] = (1-df['Lx1']/df['Lx']) 



CalculPxMethode2(df=EXPOMORTData)
EXPOMORTData

#df.expanding(..., method='table') pour travailler avec plusieurs variables

# Extrapoler quand on n'a pas assez de données

> La *table de mortalité statique*, périodique ou instantanée (ou « table de mortalité du moment ») permet de caractériser la mortalité d’une population à un moment précis, toutes générations confondues. Ce tableau suppose une certaine stabilité des décès dans le futur. 


> La *table dynamique ou prospective* (ou « table de mortalité par génération ») intègre l’âge et le temps dans la probabilité de décès. Elle est établie à partir d’une génération réelle (et non fictive comme la précédente table), observant les niveaux de mortalité réels en fonction de l’année de naissance. Si elle est plus représentative que l’autre, elle ne peut se construire qu’à la fin d’une génération ou par *extrapolation*.  
[source](https://www.wedou.fr/definitions/tables-de-mortalite#:~:text=La%20table%20de%20mortalit%C3%A9%20statique,des%20d%C3%A9c%C3%A8s%20dans%20le%20futur.)

### Diagramme de Lexis
https://en.wikipedia.org/wiki/Lexis_diagram

![image.png](attachment:image.png)


source : [Lexis Diagram](https://link.springer.com/chapter/10.1007/978-3-319-64820-0_2)



![image-3.png](attachment:image-3.png)


Quelques exemples : 
* un effet "covid" sera un effet de période, touchant l'année 2020 et 2021
* une guerre aura normalement un effet cohorte important, touchant les jeunes en âge de combattre
* améliorer le risque en mobylette aura un effet âge (réduisant fortement la mortalité des 14-25 ans)

![image-2.png](attachment:image-2.png)



en dehors de l'effet age structurel, on voit que la déformation de la mortalité se fait par cohorte.

 ### Exercice (compliqué) 10 points à se partager pour le groupe qui fait le travail (5 pts max par personne ;)
    Les diagrammes de Lexis ne sont pas réellement programmés en Python alors qu'ils ne sont en R. 
    L'idée est de pouvoir graphiquer automatiquement la fig. 2.2 en plus du graphe sous jacent (on peut le faire via matrice colorée)

In [ ]:
 
import pylexis
from plotnine import ggplot, geom_point, aes, geom_violin,theme,theme_minimal,geom_jitter,geom_segment

diagrammeLexis = pylexis.Diagram(1990, 2019, 30, 40)
diagrammeLexis.lexis_fill('cohort', 1988, "yellow")
diagrammeLexis.lexis_fill('year', 2000, "gray")


def AjouterPersonne(df=DEATHSynthese):
    df=df.reset_index()
    df = df[(df['SEXE']==1)]
    for index, row in df.iterrows():
        age= 2019-row['ANAIS']
        if age in range(30,40):diagrammeLexis.add_deaths(row['ANAIS'],2019,age,row['NombreMorts'])
AjouterPersonne()   


print(ggplot(EXPOData)
      +aes(x='ANAIS', xend=2019, y=0, yend='AGE')+
geom_segment(size=.4, colour="grey")
     )
 

### *Extrapoler* ou *Interpoler* ?

Ce qu'en pense Yann le Cun, l'un des fondateurs de l'Apprentissage profondu (deep learning)
https://www.youtube.com/watch?v=86ib0sfdFtw

> Pour opposer les deux, 
- Inter-poler, c'est quand on reste dans l'espace où on a des observations (par exemple, quelle est la probabilité de mourir dans l'année à 31.5 ans). Dans ce cas, le choix du modèle est moins important.
- Extra-poler, c'est quand on quitte l'espace où on a des observations ("quelle est la probabilité de mourir dans l'année d'une personne de 115 ans ?). Dans ce cadre, le choix du modèle est important.

## Gompetz, Makeham et Compagnie : les principaux modèles utilisés

Avant de regarder les modèles utilisés, regardons les propriétés que nous avons pu établir : 

#### Propriété 1 : 
> $q_x$ croissant avec l'âge : chaque année, notre probabilité de mourir augmente.

Cette propriété parait assez classique mais en pratique, elle n'est pas toujours respectée. Par exemple, on constate un mortalité importante des jeunes sur la route  : 
![image.png](attachment:image.png)

qui a longtemps fait une "bosse" de surmortalité autour de 18-25 ans surtout pour les hommes : 
![image-2.png](attachment:image-2.png)


Source : [Sécurité routière](https://www.onisr.securite-routiere.gouv.fr/etat-de-l-insecurite-routiere/bilans-annuels-de-la-securite-routiere/bilan-2019-de-la-securite-routiere#:~:text=La%20mortalit%C3%A9%20rapport%C3%A9e%20%C3%A0%20la,tu%C3%A9s%20par%20million%20d'habitants.)

In [ ]:
EXPOMORTData[(EXPOMORTData['SEXE']==1) & (EXPOMORTData['AGE'].between(14, 30, inclusive = True) )]


In [ ]:
ImprimeModalite('SEXE')

### Et l'on revient à Duvillard !

![image.png](attachment:image.png)

Soit $ l_x= z= 1-\frac{Kx}{y(x)}$. Le terme y(x) permet justement tenir compte de la propriété 1 en augmentant les décès avec l'âge.

### Définition du taux instantané de mortalité

#### Rappel de la définition de $q_x$

> Le taux de mortalité $q_{x,t}$ est la probabilité qu'une personne d'âge $x$ vivant au moment $t$ meurt dans l'année ($t$,$t+1$)

Dans la pratique, le passage par des modèles paramétriques fait que l'on préfère utiliser un modèle continu et non un modèle instantané.





#### Définition de la force de mortalité 
> On note la *force de mortalité* au moment $t$ pour les personnes d'age $x$ par $\mu_{x,t}$. 
On peut aussi considérer cette force de mortalité $\mu_{x,t}$ comme le *taux de mortalité instantané*. 
La probabilité qu'une personne soumise à cette force $\mu_{x,t}$ meurt dans l'intervalle (t,t+dt) est approximativement $\mu_{x,t}dt$ quand $dt$ est petit.

#### Exercice
Démontrer La probabilité qu'une personne soumise à cette force $\mu_{x,t}$ meurt dans l'intervalle (t,t+dt) est approximativement $\mu_{x,t}dt$ quand $dt$ est petit en utilisant les développement limités.


### Lien entre $\mu_{x,t}$ et $q{x,t}$

En considérant que $\mu_{x,t}dt$ est constant entre $x$ et $x+1$, on peut démontrer que : 
    $$ q_{x,t} \approx 1 - exp(-\mu_{x,t}) $$

#### Exercice
Démontrer cette approximation. 
Element de réponse en note de page [^1]. 



[^1]: Element de réponse
![image.png](attachment:image.png)

### Modèle de Gompetz

Gompertz (1825) a observé que la force de mortalité $\mu_x$ , lorsqu'elle était tracée sur l'échelle logarithmique, était
âge approximativement linéaire pendant la majeure partie de la vie adulte. 
![image.png](attachment:image.png)


Ainsi, sur les données masculines américaines en 1960, on voit que le modèle de Gompertz semble s'adapter très bien aux données
Pratiquement, $\mu_x$  augmente de façon exponentielle avec l'âge :
$$ \mu_x = Bc^{x} $$ où $B$ et $c$ sont des constantes.

* $\mu_x$ respecte notre première propriété et augmente avec l'âge.

Cependant, se pose le problème de prendre en compte les causes de décès qui ne seraient pas directement liées à l'âge.

C'est ainsi que William Makeham propose d'extrapoler $\mu_x$ aux grands âges sur la base d'une loi de Gompertz modifiée et tenant compte des causes de décès indépendantes de l'âge :
$$ \mu_x = A + Bc^{x} $$
où $A$ est le risque de mourir pour l'ensemble des causes indépendantes de l'âge.

source : [wikipedia](https://fr.wikipedia.org/wiki/Mod%C3%A8le_de_Gompertz)

In [ ]:
# Fonction permettant de grapher en scatter plot un ou deux graphes
def GraphiqueTaux(data,x1,y1,y2=None,titre=None,Varhue=None):
    f, ax = plt.subplots(figsize=(7, 5))
    sns.despine(f)
    sns.scatterplot(data=data,x=x1, y=y1,
                hue=Varhue,      
                palette="ch:r=-.2,d=.3_r" ,label=y1,
                sizes=(1, 8), linewidth=0,
                  ax=ax)
    if y2 is not None:
        sns.scatterplot(data=data,x=x1, y=y2,
                palette="ch:r=-.2,d=.3_r" ,label=y2,
                sizes=(1, 8), linewidth=0,
                  ax=ax)
    ax.set_title(titre)
    ax.legend()     




In [ ]:
GraphiqueTaux(EXPOMORTData[(EXPOMORTData['SEXE']==1)],x1='AGE',y1='Qx',titre='Qx')

en passant au log : 
    $$ log(\mu_x) = log(B) + log(C)X$$
    
soit $$ log(\mu_x) = b + cX$$ par un changement des paramêtres.
    

On peut donc estimer avec un modèle lineaire généralisé Poisson (du fait de l'indépendance entre les x,t) et avec lien *log*. C'est ce qu'on appelle le *lien canonique* pour une GLM avec une erreur de Poisson (à retenir car TRES Utilisé !).
    
  A noter que l'on peut travailler directement sur le nombre de décès en ajoutant un *offset* sur l'évolution de la population $e_{x,t}$ est une mesure de la taille moyenne de la population âgée de $x$ à la date de l'année civile t, exposition dite centrale au risque. 
    
source :[Fitting models of mortality with generalized linear and
non-linear models](http://www.macs.hw.ac.uk/~iain/research/Paper_2013.pdf)
      $$ q_{x,t} \approx 1 - exp(-\mu_{x,t}) $$

In [ ]:
EXPOMORTData['MUx'] = -np.log(-(EXPOMORTData['Qx']-1))

In [ ]:
GraphiqueTaux(EXPOMORTData[(EXPOMORTData['SEXE']==1)],x1='AGE',y1='MUx',y2='Qx',titre='Comparaison entre Qx et Mux')

On voit que $q_x$ et $\mu_x$ sont très proches quand $q_x$ est stable.

### Modèle linéaire Généralisé

Pour programmer en python un GLM : [video](https://www.youtube.com/watch?v=__oC5IRCFKI)

source : [GLM](http://www.macs.hw.ac.uk/~iain/research/Paper_2013.pdf)

In [ ]:
import statsmodels.api as sm

def CalculGLM(data,VarX,VarY,dist=sm.families.Poisson,Fonctionlien=sm.families.links.log()):
    Y = data[VarX]
    X = data[VarY]

    model = sm.GLM(Y,X,family=dist(link=Fonctionlien))
    results = model.fit()
    
    return results

In [ ]:
resultsPoisson= CalculGLM(data=EXPOMORTData[(EXPOMORTData['AGE'].between(0,100))],
         VarX='MUx',
         VarY=['AGE','SEXE'],
         dist=sm.families.Poisson,Fonctionlien=sm.families.links.log())
resultsPoisson.summary()

In [ ]:
Resultat = EXPOMORTData[(EXPOMORTData['SEXE']==1)][['MUx','AGE','SEXE']]
Resultat['MUxhat'] = resultsPoisson.predict(exog=y[['AGE','SEXE']])
print(Resultat)

In [ ]:
GraphiqueTaux(data=Resultat,x1='AGE',y1='MUx',y2='MUxhat',titre='Comparaison entre Mux observé et MUx Estimé')

In [ ]:
from statsmodels.graphics.api import abline_plot

#fig, ax = plt.subplots()
 
line_fit = sm.OLS(y['MUx'],sm.add_constant(y['MUxhat'], prepend=True)).fit()
fig=abline_plot(model_results=line_fit)

ax = fig.axes[0]
ax.scatter(y['MUxhat'], y['MUx'])
ax.margins(.1)
ax.set_title('Model Fit Plot - Données masculines seulement')
ax.set_ylabel('Observed values')
ax.set_xlabel('Fitted values');
plt.show()



Le modèle de Gompertz ne capture pas la très forte augmentation à 80 ans.

#### Exercice (compliqué) 
Montrer qu'il est équivalent d'utiliser un modèle GLM avec $Y=q_x$, une distribution binomiale et une fonction Link cloglog 
[solution](http://www.macs.hw.ac.uk/~iain/research/Paper_2013.pdf)

In [ ]:
resultsBinomialQx = CalculGLM(data=EXPOMORTData[(EXPOMORTData['AGE'].between(0,100))],
         VarX='Qx',
         VarY=['AGE','SEXE'],
         dist=sm.families.Binomial,
         Fonctionlien=sm.families.links.cloglog())
resultsBinomialQx.summary()


In [ ]:
Resultat = EXPOMORTData[(EXPOMORTData['SEXE']==1)][['Qx','AGE','SEXE']]
Resultat['Qxhat'] = resultsBinomialQx.predict(exog=y[['AGE','SEXE']])
print(Resultat)

In [ ]:
GraphiqueTaux(data=Resultat,x1='AGE',y1='Qx',y2='Qxhat',titre='Comparaison entre Qx observé et Qx Estimé')

# Modèle de Cox : intégrer l'evolution de l'espérance de vie

#### import de Lifeline
https://lifelines.readthedocs.io/en/latest/
lifelines is a complete survival analysis library, written in pure Python. 


In [ ]:
from lifelines import CoxPHFitter
from lifelines import KaplanMeierFitter

In [ ]:

from lifelines.datasets import load_rossi

rossi = load_rossi()
rossi



In [ ]:

cph = CoxPHFitter()
cph.fit(rossi, duration_col='week', event_col='arrest')

cph.print_summary()  # access the individual results using cph.summary

In [ ]:
# https://lifelines.readthedocs.io/en/latest/Survival%20Regression.html
cph = CoxPHFitter()
cph.fit(EXPOMORTData[(EXPOMORTData['AGE'].between(0,100))], duration_col='AGE', event_col='arrest')

cph.print_summary()  # access the individual results using cph.summary

# Espérances de vie

Jusqu'à 80 ans...


# Exercices


 

![image.png](attachment:image.png)

## Le viager

[source](https://fr.wikipedia.org/wiki/Jeanne_Calment)


https://fr.wikipedia.org/wiki/Jeanne_Calment#/media/Fichier:Doyens_humanit%C3%A9.png

![image.png](attachment:image.png)

Madame Jeanne Calment, 90 ans, souhaite mettre sa maison en viager : 
![image.png](attachment:image.png)

Quelle espérance de vie calculez vous ?

# Et si on ajoutait un état ?